# Divisional Demographic Analysis



The goal of this notebook is to output analyses of demographics by division and demographics. The top level folder has three subdirectories: `input`, `output` and `procedure`. `input` holds raw data, `output` holds the results of running analyses, and all analysis scripts are in `procedure`. 

This script assumes that you are running with `procedure` as your current working directory (i.e. from the `procedure` folder), such that input files will have paths like `../input/some_file.txt`. 

## Check available data files

In [ ]:
import pandas as pd
from os import listdir
from os.path import join,splitext

#set the input directory path to ../input
#in a system independent fashion
data_dir = join("..","input")
output_dir = join("..","output")

#collect the names of all files in the data directory
input_filepaths = listdir(data_dir)

print(f"Available data files: {input_filepaths}")

You should see 'student_data_spring_2023.csv` in the list of input files.

## Remove identifying data columns - only needs to be run once

Some data columns contain identifying data. We only want to work with de-identified data, so we will remove these, then save 'clean' tables for analysis 

In [ ]:
identifying_columns = ["Student_Number","Local_Phone","perm_line_1","Student_Name",\
                       "Last_Name","First_Name","Middle_Name","Suffix","Preferred_Last",
                       "Local_Zip","Birthdate","Preferred_First","Preferred_Middle","Email","ExtEmail", 'Local_Address_1',\
                       'Local_Address_2', 'Local_City', 'Local_State', 'perm_line_2', 'perm_city', 'perm_state', 'perm_zip_5', 'perm_country', 'perm_postal_cd', 'LastSchoolName',]

for input_file_name in input_filepaths:
    if not input_file_name.startswith("student_data"):
        continue
    student_data_path = join(data_dir,input_file_name)
    
    #Get the file extension (e.g. .txt, .csv, etc)
    root_file_name,file_extension = splitext(input_file_name)
    
    #Skip files that aren't csvs (note: you can export to csv from Excel)
    if file_extension != ".csv":
        print(f"Input file {input_filepath} is not a .csv file ...skipping")
    
    student_data = pd.read_csv(student_data_path)

    
    for col in identifying_columns:
        if col in student_data.columns:
            student_data = student_data.drop([col],axis=1)
        
    clean_student_data_path = join(output_dir,root_file_name + ".csv")
    
    majors = student_data['Major1'].unique()
    
    #Codes translated from course catalog at https://www.uwb.edu/registrar/autumn-2023
    
    simplified_names = {"BEARTH":"Earth Systems Science","BIMD":"Interactive Media and Design",\
                        "B CORE":"University Studies and Programs - Discovey Core",\
                        "B CUSP": "University Studies and Programs - General Education Requirements",\
                        "B DATA": "Data Studies",\
                        "BJAPAN": "Japanese",\
                        "B LEAD": "Leadership",\
                        "B SPAIN": "Spanish",\
                        "B WRIT": "Writing",\
                        "B BUS": "Business",\
                        "B MBA": "Business (MBA)",\
                        "B BECN": "Business Economics",\
                        "ELCBUS": "Business in Bellevue",\
                        "B ACCT": "Accounting",\
                        "BEDUC" : "Education",\
                        "B ED"  : "Education",\
                        "LEDE"  : "Leadership Development for Educators",\
                        "BIS"   : "Interdisciplinary Studies",\
                        "BISSKL": "Interdisciplinary Study Skills",\
                        "BISAES": "American & Ethnic Studies",\
                        "BISCLA": "Culture, Literature, and the Arts",\
                        "BES"   : "Environmental Studies",\
                        "BISGWS": "Gender, Women and Sexuality Studies",\
                        "BISGST": "Global Studies",\
                        "BISIA" :"Interdisciplinary Arts",\
                        "BISLEP":"Law, Economics, and Public Policy",\
                        "BISMCS": "Media and Communication Studies",\
                        "BISPSY": "Psychology",\
                        "B PSY" : "Psychology",\
                        "BISSTS": "Science, Technology, and Society",\
                        "BISSTA": "Social Thought and Action",\
                        "BPOLST": "Master of Policy Studies",\
                        "BCULST": "Master of Cultural Studies",\
                        "BCWRIT": "Master of Fine Arts in Creative Writing and Poetics",\
                        "BHLTH" : "Health Electives",\
                        "BHS"   : "Health Studies",\
                        "B NURS" : "Nursing",\
                        "BST"   : "Science and Technology",\
                        "B BIO" : "Biology",\
                        "CSS"   : "Computing and Software Systems",\
                        "CSSSKL": "Computing and Software Systems Skills",\
                        "MCSS"  : "Computing and Software Systems (Masters)",\
                        "B CE"  : "Computer Engineering",\
                        "B ENGR": "Engineering",\
                        "B EE"  : "Electrical Engineering",\
                        "BEE"   : "Electrical Engineering",\
                        "B MATH": "Mathematics",\
                        "STMATH": "Mathematics",\
                        "B ME"  : "Mechanical Engineering",\
                        "B CHEM": "Chemistry",\
                        "B PHYS": "Physics",\
                        "MTV":"Mathematical Thinking and Visualization"}
    
    major_mapping = {"All Majors":"All Majors"}
    for m in majors:       
        for s in simplified_names.keys():           
            if s in m:
                print(f"Shortening {m} to {simplified_names[s]}")
                major_mapping[m] = simplified_names[s]

    student_data = student_data.replace({"Major1":major_mapping,"Gender":{" ": "All Others"}})
    
    student_data.to_csv(clean_student_data_path)

## Show the format of Student Data

In [ ]:
student_data

## Show available data columns

In [ ]:
data_columns = list(student_data.columns)
print("Student data columns:", data_columns)

## Define helper functions for summarizing and graphing data

In [ ]:
import seaborn as sns
from IPython.display import display,HTML
import matplotlib.pyplot as plt
import statsmodels.stats.proportion as smprop


def summarize_demographics_by_division(student_data, demographic_col='Gender'):
    """Create a dataframe of demographics by division 
    
    student_data -- pandas dataframe of student data
    demographic col -- name of the column we want to compare (e.g. 'Gender', 'Underrepresented')
    
    """
    #Filter to just our columns of interest
    major_data = student_data.loc[:,['Major1',demographic_col]]

    #Count how often each value occurs in this reduced table
    by_major_counts = major_data.value_counts()

    #Weird reset index step -- this converts our grouped series back into a 
    #regular 'ol DataFrame 
    by_major_counts = by_major_counts.reset_index()

    #Rename columns so they are more readable
    by_major_counts.columns = ['Major1',demographic_col,'Count']
    by_major_counts
    by_major_counts = by_major_counts.sort_values(['Major1',demographic_col])

    #Now repeat to get the campus gender counts
    campus_data = student_data.loc[:,[demographic_col]]
    #Count how often each value occurs in this reduced table
    campus_counts = campus_data.value_counts()
    campus_counts = campus_counts.reset_index()
    campus_counts.columns = [demographic_col,'Count']
    campus_counts['Major1'] = 'All Majors'
    campus_counts = campus_counts.sort_values(['Major1',demographic_col])

    major_df = pd.concat([by_major_counts,campus_counts])
    majors = major_df['Major1'].unique()


    major_df[f"Count_by_Major_and_{demographic_col}"] = major_df.groupby(["Major1",demographic_col])["Count"].transform('sum')
    major_df["Count_by_Major"] = major_df.groupby(["Major1"])["Count"].transform('sum')
    major_df[f"{demographic_col}_Frequency_within_Major"] =\
    major_df[f"Count_by_Major_and_{demographic_col}"]/major_df["Count_by_Major"]
    major_df = major_df.drop("Count",axis=1)
    major_df = major_df.drop_duplicates()
    return major_df

def get_campus_fraction(df,demographic_col,focal_category,all_majors_category='All Majors',verbose=False):
    result =  get_major_fraction(df,demographic_col,focal_category,major=all_majors_category,verbose=verbose)
    return result

def get_major_fraction(df,demographic_col,focal_category,major,verbose=False):
    category_count = df[(df[demographic_col] == focal_category) & (df['Major1'] == major)][f'Count_by_Major_and_{demographic_col}'].sum()
    non_category_count = df[(df[demographic_col] != focal_category) & (df['Major1'] == major)][f'Count_by_Major_and_{demographic_col}'].sum()
    total = category_count + non_category_count
    category_fraction =\
    category_count / total
    rounded_percentage = round(category_fraction*100,1) 
    if verbose:
        print(f" {category_count} ({rounded_percentage} %) of {total} students in {major}, are listed as {demographic_col} = {focal_category}" )
    return category_count,non_category_count,category_fraction

def get_major_ratio(df,demographic_col,focal_category,major='Biology',all_majors_category='All Majors',verbose=False):
    _,_,category_fraction= get_major_fraction(df,demographic_col,focal_category,major=major,verbose=verbose)  
    _,_,campus_category_fraction = get_campus_fraction(df,demographic_col,focal_category,verbose=verbose)
    return category_fraction/campus_category_fraction

def plot_demographic_frequencies(major_df,demographic_col,output_dir="../output"):
    """Save to output"""
    
    for major in list(set(major_mapping.values())):
        print(f"Plotting major: {major}")
        major_rows = major_df['Major1'] == major 
        campus_rows = major_df['Major1'] == 'All Majors'

        rows_to_keep = major_rows | campus_rows # logical element-wise OR
        curr_data = major_df[rows_to_keep]

        unique_demographic_values = sorted(list(curr_data[demographic_col].unique()))

        
        hue_order = sorted(list(curr_data[demographic_col].unique()))
            
        palette = sns.color_palette("husl", len(unique_demographic_values))
        sns.barplot(x='Major1', y=f'{demographic_col}_Frequency_within_Major', hue=demographic_col,hue_order=hue_order,\
              data=curr_data,edgecolor="black",palette=palette)

        graph_output_file = join(output_dir,f"{major}_{demographic_col}_frequency.png")
        plt.legend(bbox_to_anchor=(1.1,1.0))
        plt.savefig(graph_output_file,bbox_inches="tight",facecolor="white")
        plt.show()
        plt.clf()
    
        
def summarize_counts_by_category(df,demographic_col):
    """Summarize counts within a focal demographic category on campus vs. within majors
    df -- a pandas DataFrame 
    demographic_col -- the demographic column to add counts for
    """
    data = []

    for focal_category in sorted(list(df[demographic_col].unique())):

        #Calculate demographics across all majors
        campus_category_count,campus_non_category_count,campus_category_fraction = get_campus_fraction(df,demographic_col,focal_category)


        df[f'Campus_{demographic_col}_count'] = campus_category_count
        df[f'Campus_non-{demographic_col}_count'] = campus_non_category_count
        df[f'Campus_{demographic_col}_fraction'] = campus_category_fraction

        for major in list(set(major_mapping.values())):

            #Calculate demographics in this major
            category_count,non_category_count,category_fraction =\
              get_major_fraction(df,demographic_col,focal_category,major=major)

            major_ratio = get_major_ratio(df,demographic_col,focal_category,major=major)

            row_data = {"Major1":major,f"{demographic_col}":focal_category,\
                        "Y":category_count,"N":non_category_count,\
                        "Percent": round(category_fraction*100,3),\
                        "Y(Campus)":campus_category_count,\
                        "N(Campus)":campus_non_category_count,\
                        "Percent(Campus)":round(campus_category_fraction*100,3),\
                        "Ratio_in_Major_vs_Campus":major_ratio
                        }
            data.append(row_data)

    graph_data = pd.DataFrame(data)
    return graph_data

def CI_from_row(row):
    """Return a 95% confidence interval from a data row"""
    count1 = row["Y"]
    nobs1 = row["Y"] + row["N"]
    count2 = row["Y(Campus)"]
    nobs2 = row["Y(Campus)"] + row["N(Campus)"]
        
    #Test 95% Confidence interval, using a method that tests the ratio of proprotions
    result = smprop.confint_proportions_2indep(count1,nobs1,count2,nobs2, compare= 'ratio')
    return result

def proportion_test_from_row(row):
    """Return p-value for a proportion test from a data row"""
    count1 = row["Y"]
    nobs1 = row["Y"] + row["N"]
    count2 = row["Y(Campus)"]
    nobs2 = row["Y(Campus)"] + row["N(Campus)"]
        
    #Test 95% Confidence interval, using a method that tests the ratio of proprotions
    result = smprop.test_proportions_2indep(count1,nobs1,count2,nobs2, \
      compare= 'ratio',alternative="two-sided")
    
    return result


def add_CI_to_graph_data(graph_data):
    """Add a confidence interval to graph data
    
    graph_data - a pandas DataFrame
    """
    
    

    result = graph_data.apply(CI_from_row,axis=1)
    CI95_low = [r[0] for r in result]
    CI95_high = [r[1] for r in result]
    graph_data["CI95_low"] = CI95_low
    graph_data["CI95_high"] = CI95_high
    
    stats_result = graph_data.apply(proportion_test_from_row,axis=1)
    stats = [s[0] for s in stats_result]
    p = [s[1] for s in stats_result]

    #Although we already have the CI
    #for graphing purposes it's more 
    #straightforward to express the 
    #length from the upper and lower CI95
    #to the mean so we can draw error bars

    graph_data["Lower_Error"] = [i for i in map(float,abs(graph_data["Ratio_in_Major_vs_Campus"] - CI95_low))]
    graph_data["Upper_Error"] = [i for i in map(float,abs(CI95_high -graph_data["Ratio_in_Major_vs_Campus"]))]
    graph_data["p (proportion test, two-sided)"] = [i for i in p]
    return graph_data

def plot_demographic_ratio_by_major(graph_data,demographic_col):
    """Plot demographic ratios by major"""
    unique_majors = sorted(list(graph_data['Major1'].unique()))

    for major in unique_majors:
        curr_graph_data = graph_data[graph_data['Major1'] == major]
        display(HTML(curr_graph_data.to_html()))
        palette = sns.color_palette("husl", len(unique_demographic_values))

        ax = sns.barplot(x='Ratio_in_Major_vs_Campus', y=f'{demographic_col}',\
            data=curr_graph_data,palette=palette,ci=None,orient="h")
        plt.title(f"{major} Demographic Ratios vs. Campus")
        plt.axvline(x=1, color='red', linestyle='--', label='Campus average')
        plt.xticks,rotation="90"
        graph_output_file = join(output_dir,f"{major}_{demographic_col}_demographic_ratios")

        errors = [curr_graph_data["Lower_Error"],curr_graph_data["Upper_Error"]]
        ax.errorbar(data=curr_graph_data, x='Ratio_in_Major_vs_Campus', y= f'{demographic_col}',\
                    xerr=errors, ls='', lw=1, color="black",capsize=5)
        ax.set_xlim(left=0,right=max(curr_graph_data['Ratio_in_Major_vs_Campus'])*1.1)
        plt.savefig(graph_output_file)
        plt.show()


## Run all analyses across demographic categories

In [ ]:
demographic_cols = ['Gender','Ethnicity_Race','Underrepresented','First_Generation', 'First_Generation_4_Yr', 'Veteran']

overall_graph_data = None
for demographic_col in demographic_cols:
  major_df = summarize_demographics_by_division(student_data,demographic_col=demographic_col)

  print(f"Generating plots for {demographic_col}")
  
  graph_data = summarize_counts_by_category(major_df,demographic_col)
  graph_data = add_CI_to_graph_data(graph_data)
  
  #Save a .csv file of raw data  
  summary_output_file = join(output_dir,f"{demographic_col}_count_summary.csv")
  #Replace blank space with underscores
  summary_output_file = "_".join(summary_output_file.split())
  print(f"Saving raw count data to {summary_output_file}")
  graph_data.to_csv(summary_output_file)
    
  #Plot frequencies and ratios vs. campus for each major
  plot_demographic_ratio_by_major(graph_data,demographic_col)
  plot_demographic_frequencies(major_df,demographic_col)
  
